# SQL: Advanced Topics

## Introduction and Table of Contents

Thus far, we have kept our discussion of what SQL actually *is* relatively brief. This has been intentional; our goal has been to get you comfortable writing SQL queries as quickly as possible so that you can use it where necessary.

In this section, we will take a moment to talk about SQL more generally and explore some more advanced topics. We will:

- Talk about databases as software, discuss SQL's formal syntax, and compare SQL with other tools.
- Spend some time reviewing more advanced SQL functionality, such as common-table expressions (CTEs) and window functions.
- Review some of the security implications related to working with SQL and revisit the concept of placeholder bindings.

We will end this section by showcasing how you can use some of these tools to answer a frequently-asked but deceptively tricky data analysis question: the "top-n cases by group" problem.

- [SQL, SQLite, and Comparisons with Other Tools](#SQL,-SQLite,-and-Comparisons-with-Other-Tools)
    - [Why SQL?](#Why-SQL?)
    - [SQLite vs. Other RDBMSs](#SQLite-vs.-Other-RDBMSs)
    - [The Structure of a `SELECT` Statement](#The-Structure-of-a-SELECT-Statement)
    - [Data Transformations in SQL vs. Pandas](#Data-Transformations-in-SQL-vs.-Pandas)
- [Advanced Functionality](#Advanced-Functionality)
    - [Dataset Overview: The NYC Jobs Postings Database](#Dataset-Overview:-The-NYC-Jobs-Postings-Database)
    - [Introduction to SQLite String Manipulation](#Introduction-to-SQLite-String-Manipulation)
    - [Introduction to Recoding Variables with CASE Statements in SQLite](#Introduction-to-Recoding-Variables-with-CASE-Statements-in-SQLite)
    - [Introduction to Inline Subqueries and Common Table Expressions (CTEs)](#Introduction-to-Inline-Subqueries-and-Common-Table-Expressions-(CTEs))
    - [Introduction to Window Functions](#Introduction-to-Window-Functions)
    - [Handling NULL Values](#Handling-NULL-Values)
- [Security Implications and Placeholder Bindings](#Security-Implications-and-Placeholder-Bindings)
    - [Queries and User Input](#Queries-and-User-Input)
    - [SQL Injection Attacks](#SQL-Injection-Attacks)
    - [Protecting Against SQL Injections](#Protecting-Against-SQL-Injections)
- [An Applied Example: Finding the Top N Cases per Group](#An-Applied-Example:-Finding-the-Top-N-Cases-per-Group)

## SQL, SQLite, and Comparisons with Other Tools

In this section, we will spend some time talking about what databases and SQL actually are. We will then take a moment to compare SQL with other similar tools.

**If you are already intimately familiar with the landscape of modern database offerings, you may be able to skip this section.** However, if you enjoy taking a step back to see the big picture, then this section is for you.


### Why SQL?

Some of you have probably been wondering: "Why do we even *need* SQL? Can't we do most of this work with other tools that we have already seen, like [Pandas](https://pandas.pydata.org/)?" To answer this question, it may be worth taking a moment to discuss what a database management system actually *is*.

There are many ways to digitally store information. However, managing this information manually can be surprisingly difficult. As the amount of information increases in scale, the task of storing it efficiently and correctly can substantially grow in complexity. It would be useful if we had some dedicated tool for managing our collection of information, or *database*.

This is where *database management systems*, or DBMSs, can become useful. In their textbook, *Fundamentals of Database Systems*, Elmasri and Navathe define a DBMS as "a computerized system that enables users to create and maintain a database." DBMSs can provide us with desirable features and abstract away complex data storage tasks which we would otherwise need to handle ourselves.

> For example, DBMSs can make it possible for multiple users to simultaneously update a database, track metadata, and handle the storage of the data files on a hard drive. Tools like Pandas and NumPy, while extremely useful, exist to solve different problems.

![](resource/asnlib/publicdata/DBMS.png)

There are many different types of DBMSs but those which organize their data into *tables* are usually *relational database management systems*, or RDBMSs.

> Technically, an RDBMS is a database management system which implements the [relational model](https://en.wikipedia.org/wiki/Relational_model). This is a more technical concept which we will not explore here, but those of you who wish to dig deeper may find the topic's Wikipedia article enlightening. For an even more in-depth discussion, see "Chapter 5: The Relational Data Model and SQL" in *Fundamentals of Database Systems*, by Ramez Elmasri and Shamkant B. Navathe (7th Edition).

[SQLite](https://www.sqlite.org/index.html), which you used in the previous parts of this notebook, is an example of an RDBMS. Generally, SQL is the language used by RDBMSs to define, query, and update the data they are managing.

> Specifically, you have been using Python to execute SQL calls. You could, however, use other languages like C or Java to execute SQL calls with SQLite (or use SQLite directly, without any other language). In fact, you could have multiple programs written in different languages interacting with *the same database* by using a DBMS and SQL. If you have never considered this before, we encourage you to pause and take a moment to reflect on the implications of being able to do this!

So, to summarize: an RDBMS helps us handle the complex task of managing a database, and we use SQL to interact with the RDBMS.

### SQLite vs. Other RDBMSs

In this class, you have been using Python's [SQLite3](https://docs.python.org/3/library/sqlite3.html#) module to interact with databases managed by [SQLite](https://www.sqlite.org/about.html). We briefly introduced SQLite in the earlier sections of this lesson. Now, we will take a moment to compare SQLite with other RDBMSs you might have to work with in the real world.

There are several SQLite-specific details which make it distinct from other RDBMSs you might find yourself working with:

- [SQLite is *self-contained*](https://www.sqlite.org/selfcontained.html): it has very few dependencies and can run on any operating system, including embedded operating systems. The entire library is contained in a single source file.
- [SQLite is *serverless*](https://www.sqlite.org/serverless.html): processes which access the database read and write directly from disk. Most database engines, by contrast, tend to implement a client-server model; there will be a separate server process which recieves requests from clients (often via TCP/IP requests) and the server will decide what to do with those requests.
- [SQLite is *zero-configuration*](https://www.sqlite.org/zeroconf.html): users do not need to run a setup procedure, assign access permissions, or manage configuration files.

These features make SQLite an excellent option for developers who want a small, fast, and local database solution. Consequently, [SQLite is the most widely deployed database engine in the world](https://www.sqlite.org/mostdeployed.html). However, there are many other great RDBMSs out there which you might need to use at some point! For reference, we have put together a table highlighting some of the differences you might encounter in other RDBMSs outside of this course.


| Engine               | Client-Server | Free              | License              | Python API             |
|----------------------|---------------|-------------------|----------------------|------------------------|
| SQLite               | No            | Yes               | Public domain        | sqlite3                |
| Oracle               | Yes           | Edition Dependent | Proprietary          | Python-oracledb        |
| Microsoft SQL Server | Yes           | Edition Dependent | Proprietary          | pyodbc, or pymssql     |
| PostgreSQL           | Yes           | Yes               | Open Source          | psycopg3               |
| MySQL                | Yes           | Yes               | GPLv2 or Proprietary | mysql-connector-python |

[SQLite's documentation](https://www.sqlite.org/whentouse.html) outlines some of its ideal use-cases, alongside instances when another tool may be a better fit.


### The Structure of a `SELECT` Statement

Despite the fact that SQL exists as an ISO standard, many SQL engine implement non-standard versions of SQL. Even SQLite implements a [slightly non-standard](https://www.sqlite.org/omitted.html) variant of the SQL standard. However, the general structure of SQL queries will tend to be the same across various SQL dialects. In this section, we will quickly review the structure of a `SELECT` statement and provide resources for you to use when writing your own queries.

SQLite recognizes [many kinds](https://www.sqlite.org/lang.html) of statements (for example, we briefly covered `INSERT` statements in the earlier sections of this lesson). `SELECT` statements can become rather complicated, so it is worth reviewing the structure of precisely what terms can be involved with them. Specifically, components of a `SELECT` statement will *always follow this order*:

1. `SELECT result-columns`
2. `FROM table-or-subquery`
3. `WHERE expr`
4. `GROUP BY expr` (and possibly `HAVING`)
5. `WINDOW window-def`

This is sometimes called the "`SELECT` core." In addition, we can add `WITH` expressions before the query, and `ORDER BY` and `LIMIT` terms to the end of the query.

> We will discuss the use of the `WITH` and `WINDOW` terms later in this notebook.

SQLite defines [the syntax of a valid SELECT statment in their documentation](https://www.sqlite.org/lang_select.html), which is displayed as a helpful flow-chart. This can be useful for debugging purposes. However, most queries can be constructed by following this process:

1. Determine what source data is required and build the table-expression for the `FROM` statement.
2. Filter the table with the `WHERE` statement.
3. `SELECT` the desired variables, optionally aggregating the results by using the `GROUP BY` term.
4. Order and limit the results with the `ORDER BY` and `LIMIT` terms.

### Data Transformations in SQL vs. Pandas

You have likely realized that we can perform many of the same operations on tables represented in Pandas or housed in a SQL database. For example, the concept of filtering a table by evaluting some conditional expression can be accomplished by using a `WHERE` clause or [boolean indexing in Pandas](https://pandas.pydata.org/docs/user_guide/indexing.html#boolean-indexing).

It may be useful to know that Pandas has compiled a [helpful reference](https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html) of how to accomplish common SQL tasks in Pandas. We encourage you to refer to it if you find one of these tools more intuitive than the other.

## Advanced Functionality

In this section, we will expand our understanding of what is possible within an SQL query. While some of these concepts are more complicated than what we have discussed thus far, these tools can be extremely powerful in the proper circumstances.  

### Dataset Overview: The NYC Jobs Postings Database

This dataset is a normalized version of the NYC Jobs Postings data. It is available on [NYC OpenData](https://opendata.cityofnewyork.us/), which is a platform for free public data published by New York City agencies and other partners. The original dataset was retreived on 08-30-2024. The most current version of the dataset can be found [here](https://data.cityofnewyork.us/City-Government/Jobs-NYC-Postings/kpav-sd4t/about_data).

The data contains information on current job postings available on the City of New York's official jobs site. It includes both internal postings for city employees and external postings available to the general public.

The dataset has been normalized and loaded into a SQLite database `nyc_jobs.db` to enable reteival with SQL. Below is a list of the tables included in the database along with a brief description of each:

- **job_post**: Contains core information about each job posting, such as the job ID, title, posting date, and the date the posting expires (post_until). This is the central table linking to other detailed tables.
  
- **agency**: Stores details about the agencies responsible for the job postings, including the agency ID and agency name.
  
- **job_title**: Provides detailed information on the job titles listed in the postings, including the title classification and civil service title.

- **salary**: Contains salary information for the job postings, such as the salary range and salary frequency (e.g., annual, hourly).

- **location**: Details the locations associated with the job postings, including the specific work location and any relevant divisions or units.

**IMPORTANT!** Please run the cells below to properly configure your notebook environment.

In [ ]:
### Global imports
import dill
from cse6040_devkit import plugins, utils

utils.add_from_file('sql_validator', plugins)
utils.add_from_file('malicious_executor', plugins)
utils.add_from_file('safe_executor', plugins)


In [ ]:
%load_ext autoreload
%autoreload 2

import sqlite3
import pandas as pd

# Path to the SQLite database file; change as needed
db = 'resource/asnlib/publicdata/nyc_jobs.db'
# Database connection
conn = sqlite3.connect(db)

### Exercise 0: (0 points)
**get_database_schema__FREE**  

**Example:** we have defined `get_database_schema__FREE` as follows:

**This is a free exercise!** 

    **Please run the test cell below to collect your FREE point!**

    The output will show the structure of the database which we will use for the following exercises.


In [ ]:
### Solution - Exercise 0  
def get_database_schema__FREE(conn):
    # Retrieve the list of tables from sqlite_master
    tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = pd.read_sql_query(tables_query, conn)

    # Prepare DataFrame to hold combined results
    schema_info = pd.DataFrame()

    # Loop through tables to get column details
    for table in tables['name']:
        # Get column details
        pragma_table_info = f"PRAGMA table_info('{table}')"
        table_info = pd.read_sql_query(pragma_table_info, conn)
        table_info['table_name'] = table  # Add table name to the DataFrame

        # Add to main DataFrame
        schema_info = pd.concat([schema_info, table_info], ignore_index=True)

    # Select and rename relevant columns
    schema_info = schema_info[['table_name', 'name', 'type']]
    schema_info.columns = ['table', 'column', 'type']

    return schema_info

### Demo function call
df_schema = get_database_schema__FREE(conn)
display(df_schema)

 
 <!-- Test Cell Boilerplate -->  
 The test cell below will always pass. Please submit to collect your free points for get_database_schema__FREE (exercise 0).
 

In [ ]:
### Test Cell - Exercise 0  


print('Passed! Please submit.')

### Introduction to SQLite String Manipulation

By now, you should be familiar with Python's ability to process and manipulate textual data. Here, we will discuss SQLite and its limitations when dealing with strings, particularly when compared to more powerful tools like Python and enterprise-grade relational databases like Oracle and SQL Server.

SQLite offers a basic set of string functions, which can make complex string operations challenging. Notably, SQLite lacks:
- Built-in support for regular expressions
- Advanced string to datetime parsing functionailty

These limitations can force you to implement more creative solutions, often involving combinations of functions like `TRIM`, `SUBSTR`, `INSTR`, and `REPLACE` to achieve results that would be straightforward in other environments. The following exercise will ask you to get creative within these constraints while extracting meaningful data from the NYC Jobs database.

### Exercise 1: (0 points)
**parse_classification**  

**Your task:** define `parse_classification` as follows:

**Activity**: Extract and Separate Title Classification Code and Description

**Inputs**: None

**Return**: `query`: a Python string containing a SQLite query. It should query the database to create a new DataFrame from the `job_title` table with the following columns:
- `title_classification`: The original `title_classification` column from the `job_title` table.
- `title_classification_desc`: The text portion of the `title_classification` with the final dash and integer removed.
- `title_classification_code`: The integer portion of the `title_classification` after the last dash.

**Requirements/steps**:
- The database table you will need is named `job_title`.
- The column you will work with is `title_classification`.

**Hint**: There are multiple ways to solve this exercise. The following SQLite functions may help, but not all are strictly necessary, [SUBSTR](https://www.sqlite.org/lang_corefunc.html#substr), [INSTR](https://www.sqlite.org/lang_corefunc.html#instr), and [REPLACE](https://sqlite.org/lang_corefunc.html#replace).


In [ ]:
### Solution - Exercise 0  
def parse_classification():
    ###
    ### YOUR CODE HERE
    ###

### Demo function call
parse_classification_query = parse_classification()
parse_classification_df = pd.read_sql_query(parse_classification_query, conn)
display(parse_classification_df.head())

 **Example**. A correct implementation should produce, for the demo, the following output:

| title_classification   | title_classification_desc   |   title_classification_code |
|:-----------------------|:----------------------------|----------------------------:|
| Competitive-1          | Competitive                 |                           1 |
| Non-Competitive-5      | Non-Competitive             |                           5 |
| Non-Competitive-5      | Non-Competitive             |                           5 |
| Competitive-1          | Competitive                 |                           1 |
| Competitive-1          | Competitive                 |                           1 |


 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for parse_classification (exercise 1). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [ ]:
### Test Cell - Exercise 0  

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    execute_tests = dill.load(f)

# Execute test
passed, test_case_vars = execute_tests(func=plugins.sql_executor(parse_classification),
              ex_name='parse_classification',
              key=b'ml_6Ers6uRKDReNOadss3eZ9fueUNRX4IImbkbFoTRw=', 
              n_iter=100)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars

assert passed, 'The solution to parse_classification did not pass the test.'

###
### AUTOGRADER TEST - DO NOT REMOVE
###
print('Passed! Please submit.')

### Introduction to Recoding Variables with CASE Statements in SQLite

In SQLite, the `CASE` statement is a powerful tool that allows you to recode variables based on specific conditions. It works similarly to an `IF-THEN-ELSE` structure in programming languages. By using `CASE`, you can create new columns, transform existing values, or recategorize data based on conditional logic directly within your SQL queries.

#### Example Case Statement

Suppose you have a table called `employees` with a column named `experience_level` that categorizes employees as `'Junior'`, `'Mid'`, or `'Senior'`. You want to create a new column called `salary_band` that assigns a salary band based on the `experience_level`. Here’s a simple example:

```sql
SELECT
    employee_id,
    experience_level,
    CASE
        WHEN experience_level = 'Junior' THEN 'Low'
        WHEN experience_level = 'Mid' THEN 'Medium'
        WHEN experience_level = 'Senior' THEN 'High'
        ELSE 'Unknown'
    END AS salary_band
FROM
    employees;
```

### Exercise 2: (0 points)
**convert_post_until**  

**Your task:** define `convert_post_until` as follows:

**Activity**: Convert Date Strings in the `post_until` Column to Standard Date Format

**Objective**: Write a SQLite query that converts the date strings in the `post_until` column from the `job_post` table into the standardized `YYYY-MM-DD` format.

**Return**: `query` — a Python string containing the SQLite query that generates a DataFrame with the following columns:
- `post_until`: The original date string from the `job_post` table.
- `post_until_converted`: The converted date string in `YYYY-MM-DD` format.

**Steps**:
- You will be working with the `job_post` table.
- The `post_until` column contains date strings in the `DD-MMM-YYYY` format (e.g., `24-AUG-2024`).
- Convert these date strings to the `YYYY-MM-DD` format (e.g., `2024-08-24`).

**Hints**:
- There are two potential approaches:
  1. Manually construct a `CASE` statement using SQLite functions like [PRINTF](https://www.sqlite.org/printf.html) and [STRFTIME](https://www.sqlite.org/lang_datefunc.html#strftime).
  2. Dynamically generate the `CASE` statement in Python using `f-strings` and `join`.


In [ ]:
### Solution - Exercise 2  
def convert_post_until():
    ###
    ### YOUR CODE HERE
    ###

### Demo function call
convert_post_until_query = convert_post_until()
convert_post_until_df = pd.read_sql_query(convert_post_until_query, conn)
display(convert_post_until_df[convert_post_until_df['post_until'].notnull()].head())

 **Example**. A correct implementation should produce, for the demo, the following output:

| post_until   | post_until_converted   |
|:-------------|:-----------------------|
| 25-AUG-2024  | 2024-08-25             |
| 24-AUG-2024  | 2024-08-24             |
| 12-JUN-2025  | 2025-06-12             |
| 05-SEP-2024  | 2024-09-05             |
| 25-SEP-2024  | 2024-09-25             |

 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for convert_post_until (exercise 2). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [ ]:
### Test Cell - Exercise 2  

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    execute_tests = dill.load(f)

# Execute test
passed, test_case_vars = execute_tests(func=plugins.sql_executor(convert_post_until),
              ex_name='convert_post_until',
              key=b'ml_6Ers6uRKDReNOadss3eZ9fueUNRX4IImbkbFoTRw=', 
              n_iter=100)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars

assert passed, 'The solution to convert_post_until did not pass the test.'

###
### AUTOGRADER TEST - DO NOT REMOVE
###
print('Passed! Please submit.')

### Introduction to Inline Subqueries and Common Table Expressions (CTEs)

When working with SQL, it's common to encounter situations where you need to perform calculations or data transformations based on grouped or filtered data. Two powerful tools for handling such scenarios are **inline subqueries** and **Common Table Expressions (CTEs)**. Both approaches allow you to structure your queries in a way that breaks down complex operations into manageable steps, but they do so in slightly different ways.

#### Inline Subqueries

An **inline subquery** is a query nested within another SQL query, typically within the `SELECT`, `WHERE`, or `FROM` clause. Inline subqueries allow you to perform calculations or filtering within the context of the outer query. They are particularly useful when you need to filter results based on aggregated values like maximum, minimum, or average.

**Example**: Suppose you have a table `employees` and you want to compare each employee's salary to the highest salary in their department:

```sql
SELECT e.name, e.department, e.salary, dept_max.max_salary
FROM employees e
JOIN (SELECT 
        department, 
        MAX(salary) AS max_salary
      FROM employees
      GROUP BY department) as dept_max ON e.department = dept_max.department;
```

This query uses an inline subquery to determine the maximum salary for each department. This intermediate result is joined to the `employees` table to attach the max department salary with each employee.

Inline subqueries are powerful, but they can become less efficient when the same calculation needs to be repeated multiple times, as each repetition can slow down the query execution. Visit the [w3resource's SQLite Subqueries page](https://www.w3resource.com/sqlite/sqlite-subqueries.php) to learn more.

#### Common Table Expressions (CTEs)

A **Common Table Expression (CTE)**, introduced by the `WITH` clause, is a named temporary result set that you can reference within your main query. CTEs improve the readability and maintainability of SQL code, especially for complex queries, by allowing you to break down operations into logical steps.

**Example**: The same result as the inline subquery example above can be achieved using a CTE:

```sql
WITH MaxSalaries AS (
    SELECT 
        department, 
        MAX(salary) AS max_salary
    FROM employees
    GROUP BY department
)
SELECT e.name, e.department, e.salary, ms.max_salary
FROM employees e
JOIN MaxSalaries ms ON e.department = ms.department
```

This CTE first calculates the maximum salary for each department, then the main query joins this result with the original `employees` table to find the employees with the highest salary in their department.

CTEs are often preferred for their clarity and efficiency, particularly when the same calculation or transformation needs to be referenced multiple times within a query. Visit the [SQLite Documentation's CTE page](https://www.sqlite.org/lang_with.html) to learn more.

### Exercise 3: (0 points)
**max_salary_by_category_inline**  

**Your task:** define `max_salary_by_category_inline` as follows:

**Activity**: Identify the Maximum Salary for Each Job Category Using an Inline Subquery

**Inputs**: None

**Return**: `query`: A Python string containing a SQLite query. It should query the database to create a new DataFrame from the `salary`, `job_posting`, and `job_title` tables with the following columns:
- `business_title`: The business title from the `job_title` table.
- `job_category`: The job category from the `job_title` table.
- `salary_range_to`: The maximum salary range for a job posting.
- `max_salary`: The maximum salary range for all job postings with the same `job_category`

**Requirements/steps**:
- JOIN the `salary`, `job_posting`, and `job_title` tables to correctly match each job posting with its corresponding title, and salary.
- Use an inline subquery to determine the maximum salary (`salary_range_to`) for each job category.
  - The query must contain an open parenthesis followed by the SELECT keyword.
  - The query must not contain the WITH keyword.
- The query should use the `DISTINCT` keyword to ensure that there are no duplicate values.

**Hint**: The following SQLite functions and concepts will help: [JOIN](https://www.sqlite.org/syntax/join-operator.html), [MAX](https://www.sqlite.org/lang_aggfunc.html#max), and [SUBQUERY](https://www.sqlite.org/lang_expr.html#subqueries).


In [ ]:
### Solution - Exercise 3  
def max_salary_by_category_inline():
    ###
    ### YOUR CODE HERE
    ###

### Demo function call
max_salary_by_category_inline_query = max_salary_by_category_inline()
max_salary_by_category_inline_df = pd.read_sql_query(max_salary_by_category_inline_query, conn)
display(max_salary_by_category_inline_df.head())

 **Example**. A correct implementation should produce, for the demo, the following output:

| business_title                                   | job_category                                      |   salary_range_to |   max_salary |
|:-------------------------------------------------|:--------------------------------------------------|------------------:|-------------:|
| SECTION SUPERVISOR                               | Administration & Human Resources                  |           66672.0 |      271736.0 |
| SECTION SUPERVISOR                               | Administration & Human Resources                  |           84276.0 |      271736.0 |
| Attorney                                         | Administration & Human Resources                  |          108156.0 |      271736.0 |
| Clerical Associate IV                            | Administration & Human Resources                  |           68645.0 |      271736.0 |
| Senior Customer Service Representative - 644504 | Administration & Human Resources                  |           51796.0 |      271736.0 |

 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for max_salary_by_category_inline (exercise 3). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [ ]:
### Test Cell - Exercise 3  

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    execute_tests = dill.load(f)

    plugin_kwargs = utils.load_object_from_publicdata('max_salary_by_category_inline_plugin_kwargs')

# Execute test
passed, test_case_vars = execute_tests(func=plugins.sql_validator(max_salary_by_category_inline, **plugin_kwargs),
              ex_name='max_salary_by_category_inline',
              key=b'ml_6Ers6uRKDReNOadss3eZ9fueUNRX4IImbkbFoTRw=', 
              n_iter=100)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars

assert passed, 'The solution to max_salary_by_category_inline did not pass the test.'

###
### AUTOGRADER TEST - DO NOT REMOVE
###
print('Passed! Please submit.')

### Exercise 4: (0 points)
**max_salary_by_category_cte**  

**Your task:** define `max_salary_by_category_cte` as follows:

**Activity**: Identify the Maximum Salary for Each Job Category Using a query with a CTE.

**Inputs**: None

**Return**: `query`: A Python string containing a SQLite query. It should query the database to create a new DataFrame from the `salary`, `job_posting`, and `job_title` tables with the following columns:
- `job_category`: The job category from the `job_title` table.
- `salary_range_to`: The maximum salary range for each job category.

**Requirements/steps**:
- JOIN the `salary`, `job_posting`, and `job_title` tables to correctly match each job posting with its corresponding title, and salary.
- Use a CTE to determine the maximum salary (`salary_range_to`) for each job category.
  - The query must contain the `WITH` keyword.
- The query should use the `DISTINCT` keyword to ensure that there are no duplicate values.

**Hint**: The following SQLite functions and concepts will help: [JOIN](https://www.sqlite.org/syntax/join-operator.html), [MAX](https://www.sqlite.org/lang_aggfunc.html#max), and [SUBQUERY](https://www.sqlite.org/lang_expr.html#subqueries).


In [ ]:
### Solution - Exercise 4  
def max_salary_by_category_cte():
    ###
    ### YOUR CODE HERE
    ###

### Demo function call
max_salary_by_category_cte_query = max_salary_by_category_cte()
max_salary_by_category_cte_df = pd.read_sql_query(max_salary_by_category_cte_query, conn)
display(max_salary_by_category_cte_df.head())

 **Example**. A correct implementation should produce, for the demo, the following output:

| business_title                                   | job_category                                      |   salary_range_to |   max_salary |
|:-------------------------------------------------|:--------------------------------------------------|------------------:|-------------:|
| SECTION SUPERVISOR                               | Administration & Human Resources                  |           66672.0 |      271736.0 |
| SECTION SUPERVISOR                               | Administration & Human Resources                  |           84276.0 |      271736.0 |
| Attorney                                         | Administration & Human Resources                  |          108156.0 |      271736.0 |
| Clerical Associate IV                            | Administration & Human Resources                  |           68645.0 |      271736.0 |
| Senior Customer Service Representative - 644504 | Administration & Human Resources                  |           51796.0 |      271736.0 |

 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for max_salary_by_category_cte (exercise 4). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [ ]:
### Test Cell - Exercise 4  

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    execute_tests = dill.load(f)

    plugin_kwargs = utils.load_object_from_publicdata('max_salary_by_category_cte_plugin_kwargs')

# Execute test
passed, test_case_vars = execute_tests(func=plugins.sql_validator(max_salary_by_category_cte, **plugin_kwargs),
              ex_name='max_salary_by_category_cte',
              key=b'ml_6Ers6uRKDReNOadss3eZ9fueUNRX4IImbkbFoTRw=', 
              n_iter=100)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars

assert passed, 'The solution to max_salary_by_category_cte did not pass the test.'

###
### AUTOGRADER TEST - DO NOT REMOVE
###
print('Passed! Please submit.')

### Introduction to Window Functions

Window functions in SQLite provide a way to perform calculations across a set of table rows that are related to the current row. Unlike aggregate functions, which return a single result for a group of rows, window functions can return multiple rows for each row in the group, providing more flexibility in your analysis. Common use cases for window functions include calculating running totals, ranking, and moving averages.

Window functions are defined using the `OVER` clause, which specifies the partitioning and ordering of rows within the window. The most commonly used window functions in SQLite include `ROW_NUMBER()`, `RANK()`, and `SUM()`. These functions can be combined with `PARTITION BY` to break the data into subsets and `ORDER BY` to define the sequence in which rows are processed.

#### Example of a Window Function

Suppose you have a table called `sales` that contains the columns `employee_id`, `sale_date`, and `amount`. You want to calculate the cumulative sales for each employee over time. Here’s a simple example using the `SUM()` window function:

```sql
SELECT
    employee_id,
    sale_date,
    amount,
    SUM(amount) OVER (PARTITION BY employee_id ORDER BY sale_date) AS cumulative_sales
FROM
    sales
ORDER BY
    employee_id, sale_date;
```

In this example:
- `SUM(amount)` calculates the running total of `amount`.
- The `OVER` clause specifies that the window function should be applied to each `employee_id` individually (`PARTITION BY employee_id`) and should consider the order of `sale_date` (`ORDER BY sale_date`).
- The result is a new column, `cumulative_sales`, that shows the cumulative sales amount for each employee over time.

Vist the the [SQLite Window Functions Page](https://www.sqlite.org/windowfunctions.html) to learn more.

### Exercise 5: (0 points)
**ranking_postings_by_classification**  

**Your task:** define `ranking_postings_by_classification` as follows:

**Activity**: Rank each available job posting within its title classification grouping, ordered by the posting's salary range.

**Inputs**: None

**Return**: `query`: A Python string containing a SQLite query. It should query the database to create a new DataFrame from the `salary`, `job_posting`, and `job_title` tables with the following columns:
- `job_id`: The id from the `job_posting` table.
- `title_classification`: The classification for each title in the `job_title` table.
- `business_title`: The business title for each title in the `job_title` table.
- `posting_updated`: The value for `posting_updated` in the `job_posting` table.
- `maximum_salary`: The value for `salary_range_to` in the `salary` table.
- `job_rank`: The ranking for each position, as defined below.

**Requirements/steps**:
- The query should join the `salary`, `job_posting`, and `job_title` tables to correctly match the job categories and their corresponding salaries.
- You will need to use the `RANK()` window function to determine the rankings for the jobs.
    - You should partition the windows by the posting's title.
    - You should order the partitions by:
        - The maximum salary (descending)
        - The posting update value (descending)
        - The job ID (ascending)

**Hint**: You will need to use the [OVER](https://sqlite.org/windowfunctions.html) keyword to properly define the windows.


In [ ]:
### Solution - Exercise 5  
def ranking_postings_by_classification():
    ###
    ### YOUR CODE HERE
    ###

### Demo function call
ranks_by_classification_query = ranking_postings_by_classification()
ranks_by_classification_df     = pd.read_sql_query(ranks_by_classification_query, conn)
display(ranks_by_classification_df.head())

 **Example**. A correct implementation should produce, for the demo, the following output:

| job_id                                      | title_classification | business_title                                        | posting_updated            |   maximum_salary |   job_rank |
|:--------------------------------------------|:---------------------|:------------------------------------------------------|:---------------------------|------------------:|-----------:|
| 637954_0be214d1d713426b837b703243b4a706     | Competitive-1         | Director of Engineering, Construction Management...   | 2024-08-14T00:00:00.000    |          235036.0 |          1 |
| 637954_1f1800ddb3da459f848a3147ee88d39c     | Competitive-1         | Director of Engineering, Construction Management...   | 2024-08-14T00:00:00.000    |          235036.0 |          2 |
| 637954_2966                                 | Competitive-1         | Director of Engineering, Construction Management...   | 2024-08-14T00:00:00.000    |          235036.0 |          3 |
| 637954_3e4dfba98f484ada80813347196eedc7     | Competitive-1         | Director of Engineering, Construction Management...   | 2024-08-14T00:00:00.000    |          235036.0 |          4 |
| 637954_51c27c951b79409dbdd5aaaaaf88adcd     | Competitive-1         | Director of Engineering, Construction Management...   | 2024-08-14T00:00:00.000    |          235036.0 |          5 |

 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for ranking_postings_by_classification (exercise 5). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [ ]:
### Test Cell - Exercise 5  

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    execute_tests = dill.load(f)

# Execute test
passed, test_case_vars = execute_tests(func=plugins.sql_executor(ranking_postings_by_classification),
              ex_name='ranking_postings_by_classification',
              key=b'ml_6Ers6uRKDReNOadss3eZ9fueUNRX4IImbkbFoTRw=', 
              n_iter=100)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars

assert passed, 'The solution to ranking_postings_by_classification did not pass the test.'

###
### AUTOGRADER TEST - DO NOT REMOVE
###
print('Passed! Please submit.')


### Handling NULL Values

There are two common situations in SQL where you are likely to encounter NULL values:
- Some table has an optional field (which is often a sign that the database is not fully [normalized](https://en.wikipedia.org/wiki/Database_normalization)).
- You perform a non-inner join, which introduces NULL values.

NULL values can cause problems when you attempt to perform certain operations. For example, adding any value to a NULL value will result in another NULL value in SQLite.

There are two main tools which SQLite provides for handling default values for NULLs: [`ifnull(X, Y)`](https://www.sqlite.org/lang_corefunc.html#ifnull) and [`coalesce(X, Y, ...)`](https://www.sqlite.org/lang_corefunc.html#coalesce). Both make it possible to provide default values for NULL instances. `coalesce()` is especially powerful, as it will return the *first non-NULL instance* it encounters in its arguments. This makes it possible to provide multiple fall-back options for NULL values.

> Recall that we have already spent some time discussing how to deal with missing values in native Python data structures, such as by using [default dictionaries](https://docs.python.org/3/library/collections.html#collections.defaultdict) and methods like [`dict.get()`](https://docs.python.org/3/library/stdtypes.html#dict.get). Pandas has a [full section of documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html) dedicated to handling missing values.

> You can see how SQLite handles operations with NULL values, alongside how other databases handle the same behavior, in their [documentation](https://www.sqlite.org/nulls.html).

> `ifnull(X, Y)` is functionally identical to `coalesce(X, Y)`. However, `coalesce()` can accept an arbitrary number of arguments greater than or equal to 2; `ifnull()`, by contrast, requires precisely 2.

Consider the following example: we want to know what skills are required for each job posting contained in the `job_posting` table. We have two columns in the table which detail job requirements:

1. `preferred_skills`, which gives us a precise description of what skills a desirable candidate will possess.
2. `minimum_qual_requirements`, which provides a more general description of what is required for the position.

The following query will attempt to display the preferred skills wherever possible. If they are not available, it will fall back to the minimum requirements. If neither are available, it will tell us that the requirements are unknown.

```sql
SELECT
  job_id,
  coalesce(
    preferred_skills,
    minimum_qual_requirements,
    'UNKNOWN!'
  ) AS job_requirements
FROM job_posting
```

### Exercise 6: (0 points)
**job_location_details**  

**Your task:** define `job_location_details` as follows:

**Activity**: Obtain details about each posting's location, using sensible defaults as defined below.

**Inputs**: None

**Return**: `query`: A Python string containing a SQLite query. It should query the database to create a new DataFrame from the `job_posting` and `location` tables with the following columns:
- `job_id`: The id from the `job_posting` table.
- `location`: The location of the posting, with null values replaced where necessary. It should be equal to:
    - `work_location_1` in the `job_posting` table, if available.
    - Otherwise, `location_name` in the `location` table, if available.
    - Otherwise, set the value to the string, `UNKNOWN`.

**Requirements/steps**:
- The query should join the `job_posting` and `location` tables with a left join to properly match the locations with the postings.
- You can use either `IFNULL()` or `COALESCE()` to solve this problem.


In [ ]:
### Solution - Exercise 6  
def job_location_details():
    ###
    ### YOUR CODE HERE
    ###

### Demo function call
job_location_details_query   = job_location_details()
job_location_details_df      = pd.read_sql_query(job_location_details_query, conn)
display(job_location_details_df.head(20))

 **Example**. A correct implementation should produce, for the demo, the following output for the first ten rows:
|    job_id | location                                              |
|----------:|:------------------------------------------------------|
|  556406_0 | 4 World Trade Center                                  |
|  639388_1 | 75-20 Astoria Blvd                                    |
|  642522_2 | 350 Jay St, Brooklyn Ny                               |
|  627775_3 | 4 World Trade Center                                  |
|  626542_4 | 470 Vanderbilt Ave                                    |
|  631653_5 | 4WTC 150 Greenwich Street, 38th FL, New York NY 10007 |
|  623231_6 | 150 William Street, New York NY                       |
|  646189_7 | 55 Water St Ny Ny                                     |
|  637846_8 | 42-09 28th Street                                     |
|  645125_9 | 30-30 Thomson Ave L I City Qns                        |
| 644065_10 | 4 World Trade Center                                  |
| 642641_11 | 55 Water Street, NY, NY                               |
| 637857_12 | 30-30 47TH AVE                                        |
| 586802_13 | 59-17 Junction Blvd Corona Ny                         |
| 632233_14 | 55 Water St Ny Ny                                     |
| 593153_15 | 59-17 Junction Blvd Corona Ny                         |
| 615463_16 | 4 World Trade Center                                  |
| 643311_17 | 255 Greenwich Street                                  |
| 616695_18 | 777 Third Avenue                                      |
| 606905_19 | 520 1St Ave., N.Y.                                    |

 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for job_location_details (exercise 6). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [ ]:
### Test Cell - Exercise 6  

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    execute_tests = dill.load(f)

# Execute test
passed, test_case_vars = execute_tests(func=plugins.sql_executor(job_location_details),
              ex_name='job_location_details',
              key=b'ml_6Ers6uRKDReNOadss3eZ9fueUNRX4IImbkbFoTRw=', 
              n_iter=100)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars

assert passed, 'The solution to job_location_details did not pass the test.'

###
### AUTOGRADER TEST - DO NOT REMOVE
###
print('Passed! Please submit.')

## Security Implications and Placeholder Bindings

In part 0, we briefly alluded to this well-known [XKCD comic on SQL injection attacks](https://xkcd.com/327/). In this section, we'll explore the focus of this comic and why the implications are so important.

### Queries and User Input

Imagine that we want to build a Python function which retrieves all of the job titles from our database belonging to a specific career-level. The career-level we are interested in could be any arbitrary value.

You might realize that we can write a function which takes an argument, `level`, and define the function so that we can specify the career-level *when the function is called*.

> Python has several different mechanisms for interpolating values into strings, such as [f-strings](https://docs.python.org/3/reference/lexical_analysis.html#f-strings) and the [.format() method](https://docs.python.org/3/library/stdtypes.html#str.format). Notice that we also define SQL queries as strings in Python. So, by using string interpolation, we can fill in the blanks of our query with whatever values we want!

Let's try doing this by using string formatting. Then, we will show why this is a bad idea.

The function **get_jobs_by_level_dangerous** is simple enough. The caller gives the desired level and we use a format string to incorporate it directly into the SQL code.  

In [ ]:
def get_jobs_by_level_dangerous(level):
    return f'SELECT * FROM job_title WHERE career_level = "{level}"'

### Demo function call
entry_level_query = get_jobs_by_level_dangerous('Entry-Level')
pd.read_sql_query(entry_level_query, conn).head()

### SQL Injection Attacks

So, why is this a bad idea?

- Consider that we are defining our SQL queries by writing **text**. This text describes *code*, which tells the computer which instructions to execute.
- However, we are describing variables in our query with **text**, too. Then, when we build the query, we are substituting part of our *code* with new *text*.

So... what would happen if the *text* we substituted into our query **was valid SQL code?**

The answer is that our query could potentially do almost *anything*. This is known as a [SQL Injection Attack](https://en.wikipedia.org/wiki/SQL_injection). Using string-formatting with Python to generate SQL queries does not protect against this.

> This is not *necessarily* dangerous. For example, if you are the only person who will ever call this function, then there is no possibility that some user will intentionally try to abuse it. However, if you allow users to interact with this function (such as by collecting their inputs in a web-form), then this approach would let users execute nearly *any* SQL statement against your database.

In this exercise, you will attempt to exploit this vulnerability and obtain more records than you might be expected to access.

### Exercise 7: (0 points)
**malicious_SQL_example**  

**Your task:** define `malicious_SQL_example` as follows:

**Activity**: Return a string value, which we will end up passing to `get_jobs_by_level_dangerous`. This string should be a value which will fetch *every* record from the `job_title` table.

**Inputs**: `None`

**Return**: `level`: a Python string which we will pass as the `level` argument to `get_jobs_by_level_dangerous`. The `level` string should be *purposefully malicious* and take advantage of SQL injection vulnerabilities, as outlined below.

**Requirements/steps**:
- You may find the examples in Wikipedia's article on [Incorrectly Structured SQL Statements](https://en.wikipedia.org/wiki/SQL_injection#Technical_implementations) useful (*hint hint!*).
- You should not be returning a full query. Only return the value for `level`, which we will use to build the full query.


In [ ]:
### Solution - Exercise 7  
def malicious_SQL_example():
    ###
    ### YOUR CODE HERE
    ###

### Demo function call
malicious_query = malicious_SQL_example()
malicious_query = get_jobs_by_level_dangerous(malicious_query)
print('Your malicious query:', malicious_query)
malicious_query_df = pd.read_sql_query(malicious_query, conn)
display(malicious_query_df.head())

 **Note**. This demo may not work correctly if you have not solved the previous exercise correctly. However, you can still pass the test cell with a proper solution, even if the demo below does not execute correctly.

**Example**. A correct implementation should produce, for the demo, the following output for the first five rows:

|   index |   id | business_title                          | civil_service_title            | title_code_no   | title_classification   | career_level              | job_category                                                                                                                          |
|--------:|-----:|:----------------------------------------|:-------------------------------|:----------------|:-----------------------|:--------------------------|:--------------------------------------------------------------------------------------------------------------------------------------|
|       0 |    1 | ADVISOR FOR STRATEGIC INITIATIVES       | ADMINISTRATIVE STAFF ANALYST ( | 1002D           | Competitive-1          | Experienced (non-manager) | Administration & Human Resources Constituent Services & Community Programs Communications & Intergovernmental Affairs Social Services |
|       1 |    2 | Deputy Commissioner, Public Information | DEPUTY COMMISSIONER (DOC)      | 95043           | Non-Competitive-5      | Executive                 | Communications & Intergovernmental Affairs                                                                                            |
|       2 |    3 | Media Services Aide                     | COMMUNITY ASSISTANT            | 56056           | Non-Competitive-5      | Entry-Level               | Administration & Human Resources Technology, Data & Innovation                                                                        |
|       3 |    4 | DATA CONTROL SUPERVISOR                 | PRINCIPAL ADMINISTRATIVE ASSOC | 10124           | Competitive-1          | Entry-Level               | Administration & Human Resources Social Services                                                                                      |
|       4 |    5 | UNIT CLERK                              | CLERICAL ASSOCIATE             | 10251           | Competitive-1          | Experienced (non-manager) | Administration & Human Resources Social Services                                                                                      |

 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for malicious_SQL_example (exercise 7). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [ ]:
### Test Cell - Exercise 7  

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    execute_tests = dill.load(f)

# Execute test
passed, test_case_vars = execute_tests(func=plugins.malicious_executor(malicious_SQL_example),
              ex_name='malicious_SQL_example',
              key=b'ml_6Ers6uRKDReNOadss3eZ9fueUNRX4IImbkbFoTRw=', 
              n_iter=100)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars

assert passed, 'The solution to malicious_SQL_example did not pass the test.'

###
### AUTOGRADER TEST - DO NOT REMOVE
###
print('Passed! Please submit.')

### Protecting Against SQL Injections

This is clearly a serious security concern! What can we do about it?

> You should now have enough context to understand why this [XKCD comic](https://xkcd.com/327/) is both funny and insightful; the school-system did not know about injection attacks, and their **entire database was deleted** as a result.

Instead of directly interpolating the values ourselves, we can ask the SQLite API to do it for us. All major Python-SQL interfaces will provide some mechanism for specifying placeholders in a query. Then, the SQL library will automatically escape special characters and keep your queries from executing unwanted code.

In other words, instead of building the query *ourselves*, we will leave a placeholder in the query and give the SQL API a collection of values to interpolate. We will let the library take care of everything else.

Python's [SQLite3 Module documentation](https://docs.python.org/3/library/sqlite3.html#how-to-use-placeholders-to-bind-values-in-sql-queries) contains examples of how to do this. There are several approaches developers can use, but we will encourage the use of the `?` syntax here.

> The exact method of doing this will vary by SQL backend, so always check the documentation to see how different engines require you to do this. For example, if you are using PostgreSQL, the [Psycopg documentation](https://www.psycopg.org/psycopg3/docs/basic/params.html) will tell you the specific syntax. Tools like [Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html) will often have a specific function argument for you to use, such as `params`.

### Exercise 8: (0 points)
**get_jobs_by_level_safe**  

**Your task:** define `get_jobs_by_level_safe` as follows:

**Activity**: Return a query which will select every record from the `job_title` table,
  as long as the `career_level` column is equal to the value specified by `level`. Use placeholders
  for the variable arguments and return the variables as a tuple.

**Inputs**: `level`: A value for `career_level`, as a Python string. We will use it to decide which records to keep.

**Return**: `output`: A tuple with two elements:
- `jobs_by_level_safe_query`: A query, as a Python string, as defined above. Make sure you are using SQL placeholders!
- `params`: A tuple with one element: `level`. We will use this to correctly bind the variables to the query.

**Requirements/steps**:
- Make sure you are using placeholder syntax! You may find the documentation, linked above, useful!


In [ ]:
### Solution - Exercise 8  
def get_jobs_by_level_safe(level):
    ###
    ### YOUR CODE HERE
    ###

### Demo function call
query, params = get_jobs_by_level_safe('Entry-Level')
get_jobs_by_level_safe_df = pd.read_sql_query(query, conn, params=params)
display(get_jobs_by_level_safe_df.head())

 **Example**. A correct implementation should produce, for the demo, the following output for the first five rows:

|   index |   id | business_title                                               | civil_service_title            |   title_code_no | title_classification   | career_level   | job_category                                                   |
|--------:|-----:|:-------------------------------------------------------------|:-------------------------------|----------------:|:-----------------------|:---------------|:---------------------------------------------------------------|
|       2 |    3 | Media Services Aide                                          | COMMUNITY ASSISTANT            |           56056 | Non-Competitive-5      | Entry-Level    | Administration & Human Resources Technology, Data & Innovation |
|       3 |    4 | DATA CONTROL SUPERVISOR                                      | PRINCIPAL ADMINISTRATIVE ASSOC |           10124 | Competitive-1          | Entry-Level    | Administration & Human Resources Social Services               |
|      15 |   16 | Claim Specialist                                             | CLAIM SPECIALIST               |           30726 | Competitive-1          | Entry-Level    | Legal Affairs                                                  |
|      19 |   20 | City Mortuary Technician                                     | City Mortuary Technician       |           52020 | Labor-3                | Entry-Level    | Health                                                         |
|      34 |   35 | Public Health Assistant (Part-Time), Bureau of School Health | PUBLIC HEALTH ASSISTANT (SCHOO |           81815 | Competitive-1          | Entry-Level    | Health                                                         |

 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for get_jobs_by_level_safe (exercise 8). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [ ]:
### Test Cell - Exercise 8  

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    execute_tests = dill.load(f)

# Execute test
passed, test_case_vars = execute_tests(func=plugins.safe_executor(get_jobs_by_level_safe),
              ex_name='get_jobs_by_level_safe',
              key=b'ml_6Ers6uRKDReNOadss3eZ9fueUNRX4IImbkbFoTRw=', 
              n_iter=100)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars

assert passed, 'The solution to get_jobs_by_level_safe did not pass the test.'

###
### AUTOGRADER TEST - DO NOT REMOVE
###
print('Passed! Please submit.')

Congratulations! You should now understand one of the most common security vulnerabilities to consider when writing applications with SQL. Please remember to sanitize your user inputs when writing applications so that you don't fall victim to Bobby Tables!

## An Applied Example: Finding the Top N Cases per Group

In this section, you will use what you have learned to answer a commonly asked data analytics question: the "top n-cases per group" problem.

To begin, consider a simple example. Imagine that we wanted to filter our records to keep only the top 200 job postings, ordered by their salary. At this point, we know we can accomplish this with the following keywords:

- `ORDER BY`
- `LIMIT`

Now, consider a seemingly related problem: what if we want to keep the top 3 job postings by salary for *each group* of title classifications, as provided by the `job_title` table? Intuitively, you might recognize that you need to *partition* the table and rank the elements. We can do this with window functions. If we do this correctly, we can keep an arbitrary number of these records; we will generalize this by saying we want `n` records. You should recognize that this will mean we need to build the query with placeholder bindings. Finally, you may realize that you will need to filter the table based on the rankings: this means you will need to calculate the rankings *first* and then filter the table accordingly. This is a great indication that a CTE would make the query easier to write.

In the exercise below, you will attempt to write a query like this.

### Exercise 9: (0 points)
**top_internal_postings_by_classification**  

**Your task:** define `top_internal_postings_by_classification` as follows:

**Activity**: Get the Freshest, Highest Paying Job by Ranking and Filtering Internal Job Postings by Classification and Posting Date

**Inputs**: `n`: The number of records to keep, per group.

**Return**: 
- `query`: A Python string containing a SQLite query. It should query the database to create a new DataFrame from the `job_posting`, `job_title`, and `salary` tables with the following columns:
    - `title_classification`: The job classification title from the `job_title` table.
    - `business_title`: The business title of the job from the `job_title` table.
    - `posting_updated`: The date when the job posting was last updated.
    - `salary_range_to`: The maximum salary range for the job posting.
    - `job_rank`: The rank of the job posting within its classification based on the `posting_updated` date, salary, and job ID.
- `params`: A tuple containing `n`.

**Requirements/steps**:
- Use a Common Table Expression (CTE) to rank job postings within each `title_classification` based on the `posting_updated` date, with ties broken by `salary_range_to` and `job_id` in that order. SQLite does not allow the direct use of window functions like `RANK()` in the `WHERE` clause. Therefore, a CTE is used to first calculate the ranks and the ranks are filtered in the outer query.
- Use the `PARTITION BY` clause within the `RANK()` function to group the job postings by `title_classification` before ranking them, ensuring that the ranking is applied separately within each classification.
- Filter the results to include only job postings where `posting_type` is `'Internal'`. This is necessary to avoid duplicates that occur when both internal and external versions of a job posting exist.
- Return only the top 3 job postings per `title_classification`, ranked by their `posting_updated` date.
- Order the final results by `title_classification` and the ranking.

**Hint**: The following SQLite concepts and functions will help: [Common Table Expression (CTE)](https://www.sqlite.org/lang_with.html) as well as `RANK` and `PARTITION BY` at the [SQLite Window Functions Page](https://www.sqlite.org/windowfunctions.html).


In [ ]:
### Solution - Exercise 9  
def top_internal_postings_by_classification(n):
    ###
    ### YOUR CODE HERE
    ###

### Demo function call
top_internal_postings_by_classification_query, top_internal_postings_by_classification_params = top_internal_postings_by_classification(3)
top_internal_postings_by_classification_df = pd.read_sql_query(top_internal_postings_by_classification_query, conn, params=top_internal_postings_by_classification_params)
display(top_internal_postings_by_classification_df.head(20))

 **Example**. A correct implementation should produce, for the demo, the following output:
| title_classification     | business_title                                                                   | posting_updated         |   salary_range_to |   job_rank |
|:-------------------------|:---------------------------------------------------------------------------------|:------------------------|------------------:|-----------:|
| Competitive-1            | Traffic Analyst - OLS                                                            | 2024-08-26T00:00:00.000 |             96395 |          1 |
| Competitive-1            | Childcare Inspection Supervisor                                                  | 2024-08-26T00:00:00.000 |             84616 |          2 |
| Competitive-1            | Associate Project Manager                                                        | 2024-08-26T00:00:00.000 |             84401 |          3 |
| Exempt-4                 | Assistant District Attorney Fall 2025                                            | 2024-08-21T00:00:00.000 |            100000 |          1 |
| Exempt-4                 | Assistant District Attorney - Animal Cruelty Prosecution Initiative              | 2024-08-20T00:00:00.000 |            175000 |          2 |
| Exempt-4                 | Assistant Corporation Counsel Mayor s Office of Special Enforcement              | 2024-08-15T00:00:00.000 |            212614 |          3 |
| Labor-3                  | ELEVATOR DISPATCHER                                                              | 2024-08-23T00:00:00.000 |             50569 |          1 |
| Labor-3                  | CARETAKER X                                                                      | 2024-08-19T00:00:00.000 |             50569 |          2 |
| Labor-3                  | CARETAKER X                                                                      | 2024-08-19T00:00:00.000 |             50569 |          3 |
| Non-Competitive-5        | General Counsel                                                                  | 2024-08-26T00:00:00.000 |            150000 |          1 |
| Non-Competitive-5        | Senior Director of Research & Evaluation, Bureau of Brooklyn Neighborhood Health | 2024-08-26T00:00:00.000 |            136303 |          2 |
| Non-Competitive-5        | Chief Employer Engagement Officer                                                | 2024-08-26T00:00:00.000 |            120000 |          3 |
| Pending Classification-2 | Analyst FEMA Capital Budget Management                                           | 2024-08-22T00:00:00.000 |             74893 |          1 |
| Pending Classification-2 | Receptionist                                                                     | 2024-08-22T00:00:00.000 |             71635 |          2 |
| Pending Classification-2 | Chauffeur - Attendant                                                            | 2024-08-21T00:00:00.000 |             85000 |          3 |

 ---
 <!-- Test Cell Boilerplate -->  
The cell below will test your solution for top_internal_postings_by_classification (exercise 9). The testing variables will be available for debugging under the following names in a dictionary format.  
- `input_vars` - Input variables for your solution.   
- `original_input_vars` - Copy of input variables from prior to running your solution. Any `key:value` pair in `original_input_vars` should also exist in `input_vars` - otherwise the inputs were modified by your solution.  
- `returned_output_vars` - Outputs returned by your solution.  
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 


In [ ]:
### Test Cell - Exercise 9  

# Load testing utility
with open('resource/asnlib/publicdata/execute_tests', 'rb') as f:
    execute_tests = dill.load(f)

# Execute test
passed, test_case_vars = execute_tests(func=plugins.safe_executor(top_internal_postings_by_classification),
              ex_name='top_internal_postings_by_classification',
              key=b'ml_6Ers6uRKDReNOadss3eZ9fueUNRX4IImbkbFoTRw=', 
              n_iter=100)
# Assign test case vars for debugging
input_vars, original_input_vars, returned_output_vars, true_output_vars = test_case_vars

assert passed, 'The solution to top_internal_postings_by_classification did not pass the test.'

###
### AUTOGRADER TEST - DO NOT REMOVE
###
print('Passed! Please submit.')